<a href="https://colab.research.google.com/github/CAIKAYUBAKA/hds5210-2023/blob/main/midterm/midterm-2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """(str, int, int, float, int, int, float, str, str, str)-->int
    This function calculates the PRIEST Clinical Severity Score and return the risk percentage.
    Returns:
        float or None: Risk percentage between 0 and 1. Returns None for invalid inputs.

>>> priest('male', 30, 10, 0.96, 70, 120, 37.0, 'alert', 'air', 'unrestricted normal activity')
0.59
>>> priest('female', 70, 15, 0.91, 85, 140, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
0.99
>>> priest('unknown', 50, 20, 0.94, 60, 100, 36.8, 'confused or not alert', 'supplemental oxygen', 'limited activity, can self-care')
None
"""

    #sex to lowercase for case insensitivity
    sex = sex.lower()

    if sex not in ['male', 'female']:
        return None

    points = 0
    if sex == 'male':
        points += 1

    if 50 <= age <= 65:
        points += 2
    elif 66 <= age <= 80:
        points += 3
    elif age > 80:
        points += 4

    if respiratory_rate < 9:
        points += 3
    elif 9 <= respiratory_rate <= 11:
        points += 1
    elif 21 <= respiratory_rate <= 24:
        points += 2
    elif respiratory_rate > 24:
        points += 3

    if 17 <= points <= 25:
        return 0.59
    elif points >= 26:
        return 0.99
    else:
        return points / 100


In [2]:
print(priest('male', 30, 10, 0.96, 70, 120, 37.0, 'alert', 'air', 'unrestricted normal activity'))  # Output: 0.59
print(priest('female', 70, 15, 0.91, 85, 140, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care'))  # Output: 0.99
print(priest('unknown', 50, 20, 0.94, 60, 100, 36.8, 'confused or not alert', 'supplemental oxygen', 'limited activity, can self-care'))  # Output: None

0.02
0.03
None


In [3]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest('male', 30, 10, 0.96, 70, 120, 37.0, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.59
**********************************************************************
File "__main__", line 7, in NoName
Failed example:
    priest('male', 30, 10, 0.96, 70, 120, 37.0, 'alert', 'air', 'unrestricted normal activity')
Expected:
    0.59
Got:
    0.02
Trying:
    priest('female', 70, 15, 0.91, 85, 140, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.99
**********************************************************************
File "__main__", line 9, in NoName
Failed example:
    priest('female', 70, 15, 0.91, 85, 140, 38.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expected:
    0.99
Got:
    0.03
Trying:
    priest('unknown', 50, 20, 0.94, 60, 100, 36.8, 'confused or not alert', 'supplemental oxygen', 'limited activity, can self-care')
Expecting:
   

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [4]:
import requests


def find_hospital(age, sex, risk):
    """ (int, str, float)
    This function will find the appropriate hospital based on age, sex, and risk percentage.
    Returns:
        Name of the hospital.(str) &
        None if anything fails.
    """
    # Ensure sex is lowercase
    sex = sex.lower()


    # Make the API call
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"


    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses


        # Parse JSON response
        hospital_info = response.json()
        return hospital_info.get('hospital')


    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None


In [5]:
print(find_hospital(60, 'female', 0.4))
print(find_hospital(40, 'male', 0.1))
print(find_hospital(70, 'female', 0.8))

Emory Dunwoody Medical Center
Southwest Hospital and Medical Center
Wesley Woods Geriatric Hospital


In [6]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [7]:
import requests

def find_hospital(age, sex, risk):
    """ (int, str, bool)
    This function finds the appropriate hospital based on age, sex, and risk percentage.
    Returns:
        str: Name of the recommended hospital, or None if anything fails.
    """
    # Ensure sex is lowercase
    sex = sex.lower()

    # Make the API call
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse JSON response
        hospital_info = response.json()
        return hospital_info.get('hospital')

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Test cases
result_1 = find_hospital(35, 'male', 0.1)
result_2 = find_hospital(65, 'female', 0.9)
result_3 = find_hospital(85, 'male', 0.99)




In [8]:
print(f"Test Case 1: Expected 'Southwest Hospital and Medical Center', Got: {result_1}")
print(f"Test Case 2: Expected 'Northeast Medical Center', Got: {result_2}")
print(f"Test Case 3: Expected 'Northwest General Hospital', Got: {result_3}")


Test Case 1: Expected 'Southwest Hospital and Medical Center', Got: Southwest Hospital and Medical Center
Test Case 2: Expected 'Northeast Medical Center', Got: Wesley Woods Geriatric Hospital
Test Case 3: Expected 'Northwest General Hospital', Got: Wesley Woods Geriatric Hospital


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [9]:
def process_people(file_url):
    response = requests.get(file_url)

    if response.status_code == 200:
        try:
            data = response.json()
            print(f"Data: {data}")
        except Exception as e:
            print(f"Error parsing JSON: {e}")
import requests

# Define find_hospital and get_address functions here

# Define priest_score calculation here

def process_people(file_url):
    """ This function processes patient data and calculates PRIEST scores for each patient, returning a
        list of dictionaries containing patient information and calculated PRIEST scores.
    """
    response = requests.get(file_url)

    if response.status_code == 200:
        data = response.json()
        processed_data = []

        for row in data:
            patient_info = {}
            for key, value in row.items():
                patient_info[key] = value

            # Calculate or retrieve priest_score
            priest_score = calculate_priest_score(patient_info)  # Example function call

            if priest_score is not None:
                if priest_score < 0.1:
                    patient_info['Hospital'] = 'Home'
                    patient_info['Address'] = 'Home'
                else:
                    patient_info['Hospital'] = find_hospital(
                        int(patient_info['Age']),
                        patient_info['Sex'],
                        priest_score
                    )
                    patient_info['Address'] = get_address(patient_info['Hospital'])

            processed_data.append(patient_info)

        return processed_data

    return None

processed_data = process_people('https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-')


AttributeError: ignored

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---